# Sending and Receiving Packets

In [1]:
from scapy.all import *

In [2]:
conf.verb=1
conf.color_theme = RastaTheme()

## Stacking Layers
The / operator has been used as a composition operator between two layers. When doing so, the lower layer can have one or more of its defaults fields overloaded according to the upper layer. (You still can give the value you want). A string can be used as a raw layer.
<img src="http://scapy.readthedocs.io/en/latest/_images/fieldsmanagement.png" />

In [3]:
IP()

<IP  |>

In [4]:
IP()/TCP()

<IP  frag=0 proto=tcp |<TCP  |>>

In [5]:
Ether()/IP()/TCP()

<Ether  type=IPv4 |<IP  frag=0 proto=tcp |<TCP  |>>>

In [6]:
IP()/TCP()/"GET / HTTP/1.0\r\n\r\n"

<IP  frag=0 proto=tcp |<TCP  |<Raw  load='GET / HTTP/1.0\r\n\r\n' |>>>

In [7]:
Ether()/IP()/IP()/UDP()

<Ether  type=IPv4 |<IP  frag=0 proto=ipencap |<IP  frag=0 proto=udp |<UDP  |>>>>

In [8]:
IP(proto=55)/TCP(dport=22)

<IP  frag=0 proto=55 |<TCP  dport=ssh |>>

## Build ARP packets
- build ARP packet which has two layers - Ether and ARP

In [9]:
ls(Ether)

dst        : DestMACField                        = (None)
src        : SourceMACField                      = (None)
type       : XShortEnumField                     = (36864)


In [10]:
ls(ARP)

hwtype     : XShortField                         = (1)
ptype      : XShortEnumField                     = (2048)
hwlen      : FieldLenField                       = (None)
plen       : FieldLenField                       = (None)
op         : ShortEnumField                      = (1)
hwsrc      : MultipleTypeField                   = (None)
psrc       : MultipleTypeField                   = (None)
hwdst      : MultipleTypeField                   = (None)
pdst       : MultipleTypeField                   = (None)


In [11]:
arpPkt = Ether()/ARP()

In [12]:
arpPkt.show()

###[ Ethernet ]### 
  dst= 52:54:00:12:35:00
  src= 08:00:27:f7:e8:f7
  type= ARP
###[ ARP ]### 
     hwtype= 0x1
     ptype= IPv4
     hwlen= None
     plen= None
     op= who-has
     hwsrc= 08:00:27:f7:e8:f7
     psrc= 192.168.100.4
     hwdst= 00:00:00:00:00:00
     pdst= 0.0.0.0



In [13]:
# Change Ethernet protocol's destination field to broadcase MAC address
arpPkt['Ethernet'].dst= 'FF:FF:FF:FF:FF:FF'

In [14]:
arpPkt.show()

###[ Ethernet ]### 
  dst= FF:FF:FF:FF:FF:FF
  src= 08:00:27:f7:e8:f7
  type= ARP
###[ ARP ]### 
     hwtype= 0x1
     ptype= IPv4
     hwlen= None
     plen= None
     op= who-has
     hwsrc= 08:00:27:f7:e8:f7
     psrc= 192.168.100.4
     hwdst= 00:00:00:00:00:00
     pdst= 0.0.0.0



In [15]:
# change ARP protocol's hwdst to broadcast MAC address
#arpPkt['Ethernet']['ARP'].hwdst = 'FF:FF:FF:FF:FF:FF'
arpPkt['ARP'].hwdst = 'FF:FF:FF:FF:FF:FF'

In [16]:
arpPkt.show()

###[ Ethernet ]### 
  dst= FF:FF:FF:FF:FF:FF
  src= 08:00:27:f7:e8:f7
  type= ARP
###[ ARP ]### 
     hwtype= 0x1
     ptype= IPv4
     hwlen= None
     plen= None
     op= who-has
     hwsrc= 08:00:27:f7:e8:f7
     psrc= 192.168.100.4
     hwdst= FF:FF:FF:FF:FF:FF
     pdst= 0.0.0.0



## Capture packet using wireshark
- ARP packet is ready to be sent
- Open Wireshark and start capturing traffic

# Send packets - one way
- sendp() - works at layer two; can send ARP, Ethernet, etc.
- send() - send packets at layer 3 (IP, TCP, HTTP, etc.); will handle routing and layer 2
    - if return_packets=True is passed as parameter, these functions will also return received packes

In [17]:
help(send)

Help on function send in module scapy.sendrecv:

send(x, inter=0, loop=0, count=None, verbose=None, realtime=None, return_packets=False, socket=None, *args, **kargs)
    Send packets at layer 3
    send(packets, [inter=0], [loop=0], [count=None], [verbose=conf.verb], [realtime=None], [return_packets=False],  # noqa: E501
         [socket=None]) -> None



In [18]:
help(sendp)

Help on function sendp in module scapy.sendrecv:

sendp(x, inter=0, loop=0, iface=None, iface_hint=None, count=None, verbose=None, realtime=None, return_packets=False, socket=None, *args, **kargs)
    Send packets at layer 2
    sendp(packets, [inter=0], [loop=0], [iface=None], [iface_hint=None], [count=None], [verbose=conf.verb],  # noqa: E501
          [realtime=None], [return_packets=False], [socket=None]) -> None



In [19]:
# send the arpPkt - use sendp() because ARP is layer 2 packet
sendp(arpPkt)


Sent 1 packets.


In [20]:
! arp -n

Address                  HWtype  HWaddress           Flags Mask            Iface
192.168.100.3            ether   08:00:27:cd:17:be   C                     eth0
192.168.100.1            ether   52:54:00:12:35:00   C                     eth0
192.168.100.5            ether   08:00:27:ce:4f:bd   C                     eth0


In [21]:
# one liner
sendp(Ether(dst="ff:ff:ff:ff:ff:ff",src="00:11:22:aa:bb:cc")/ARP(hwsrc="00:11:22:aa:bb:cc",pdst="192.168.231.2"))


Sent 1 packets.


## Sending HTTP request

In [23]:
httpPkt = IP(dst="example.com")/TCP(dport=80)/"GET /index.html HTTP/1.1\r\n\r\n"

In [24]:
bytes(httpPkt)

b'E\x00\x00D\x00\x01\x00\x00@\x06 ,\xc0\xa8d\x04]\xb8\xd8"\x00\x14\x00P\x00\x00\x00\x00\x00\x00\x00\x00P\x02 \x00s\x1e\x00\x00GET /index.html HTTP/1.1\r\n\r\n'

In [25]:
hexdump(httpPkt)

0000  45 00 00 44 00 01 00 00 40 06 20 2C C0 A8 64 04  E..D....@. ,..d.
0010  5D B8 D8 22 00 14 00 50 00 00 00 00 00 00 00 00  ].."...P........
0020  50 02 20 00 73 1E 00 00 47 45 54 20 2F 69 6E 64  P. .s...GET /ind
0030  65 78 2E 68 74 6D 6C 20 48 54 54 50 2F 31 2E 31  ex.html HTTP/1.1
0040  0D 0A 0D 0A                                      ....


In [26]:
httpPkt.show()

###[ IP ]### 
  version= 4
  ihl= None
  tos= 0x0
  len= None
  id= 1
  flags= 
  frag= 0
  ttl= 64
  proto= tcp
  chksum= None
  src= 192.168.100.4
  dst= Net('example.com')
  \options\
###[ TCP ]### 
     sport= ftp_data
     dport= http
     seq= 0
     ack= 0
     dataofs= None
     reserved= 0
     flags= S
     window= 8192
     chksum= None
     urgptr= 0
     options= []
###[ Raw ]### 
        load= 'GET /index.html HTTP/1.1\r\n\r\n'



## Send and Receive Packets
- Send and receive functions are the heart of Scapy
- sr() - send L3 packets and receive answers; returns answered and unanswered packets
- sr1() - returns only one packet that answered the sent packet
- sr() and sr1() functions can only send layer 3 L3 packets (IP, HTTP, etc.)
- srp() - send and receive L2 packets (ARP, Ehternet, 802.3, etc)
- srp1() - send and receive one layer 2 packet

In [27]:
response = sr(httpPkt)

Begin emission:
Finished sending 1 packets.

Received 2 packets, got 1 answers, remaining 0 packets


In [28]:
response

(<Results: TCP:1 UDP:0 ICMP:0 Other:0>,
 <Unanswered: TCP:0 UDP:0 ICMP:0 Other:0>)

In [29]:
response[0].show()

0000 IP / TCP 192.168.100.4:ftp_data > 93.184.216.34:http S / Raw ==> IP / TCP 93.184.216.34:http > 192.168.100.4:ftp_data SA / Padding


In [30]:
response[0].summary()

IP / TCP 192.168.100.4:ftp_data > 93.184.216.34:http S / Raw ==> IP / TCP 93.184.216.34:http > 192.168.100.4:ftp_data SA / Padding


## TCP/IP full three-way handshake

### send SYN packet

In [31]:
# Create syn packet
syn = IP(dst='example.com')/TCP(dport=80, flags='S')

In [32]:
syn

<IP  frag=0 proto=tcp dst=Net('example.com') |<TCP  dport=http flags=S |>>

In [33]:
# Receive SYN-ACK packet from google.com
syn_ack = sr1(syn)

Begin emission:
Finished sending 1 packets.

Received 1 packets, got 1 answers, remaining 0 packets


In [34]:
syn_ack.show()

###[ IP ]### 
  version= 4
  ihl= 5
  tos= 0x0
  len= 44
  id= 7331
  flags= 
  frag= 0
  ttl= 255
  proto= tcp
  chksum= 0x44a1
  src= 93.184.216.34
  dst= 192.168.100.4
  \options\
###[ TCP ]### 
     sport= http
     dport= ftp_data
     seq= 157673
     ack= 1
     dataofs= 6
     reserved= 0
     flags= SA
     window= 32768
     chksum= 0x553e
     urgptr= 0
     options= [('MSS', 1460)]
###[ Padding ]### 
        load= '\x00\x00'



In [35]:
# Send Get request
getStr = 'GET / HTTP1.1\r\nHost:example.com\r\n\r\n'
request = IP(dst='example.com') / TCP(dport=80, sport=syn_ack[TCP].dport, 
                                        seq=syn_ack[TCP].ack, ack=syn_ack[TCP].seq+1, 
                                        flags='A') / getStr
reply = sr1(request)

Begin emission:
Finished sending 1 packets.

Received 1 packets, got 1 answers, remaining 0 packets


In [36]:
reply.show()

###[ IP ]### 
  version= 4
  ihl= 5
  tos= 0x0
  len= 40
  id= 7333
  flags= 
  frag= 0
  ttl= 255
  proto= tcp
  chksum= 0x44a3
  src= 93.184.216.34
  dst= 192.168.100.4
  \options\
###[ TCP ]### 
     sport= http
     dport= ftp_data
     seq= 157674
     ack= 36
     dataofs= 5
     reserved= 0
     flags= RA
     window= 32768
     chksum= 0x6cd4
     urgptr= 0
     options= []
###[ Padding ]### 
        load= '\x00\x00\x00\x00\x00\x00'



In [37]:
reply.summary()

'IP / TCP 93.184.216.34:http > 192.168.100.4:ftp_data RA / Padding'

In [38]:
tcpip = IP(dst="example.com") / TCP(dport=80) # tcp packet needs dest port
reply = sr(tcpip)
reply

Begin emission:
Finished sending 1 packets.

Received 1 packets, got 1 answers, remaining 0 packets


(<Results: TCP:1 UDP:0 ICMP:0 Other:0>,
 <Unanswered: TCP:0 UDP:0 ICMP:0 Other:0>)

In [39]:
ans, unans = reply 

In [40]:
ans.summary()

IP / TCP 192.168.100.4:ftp_data > 93.184.216.34:http S ==> IP / TCP 93.184.216.34:http > 192.168.100.4:ftp_data SA / Padding


In [41]:
unans

<Unanswered: TCP:0 UDP:0 ICMP:0 Other:0>

## TCP Syn Flooding
-  Not quite working with Scapy but should world with netwox
- form of DoS attack where attacker doesn't complete 3-way handshake by not sending ACK packet after receiving SYN-ACK from server
- attacker typically floods the victim's half-opened connections
- when queue is full, victim cannot take any more new connection
- See Taks 1 here: http://www.cis.syr.edu/~wedu/seed/Labs_16.04/Networking/TCP_Attacks/TCP_Attacks.pdf
- queue size can be viwed by using the following command

In [42]:
# on a server VM, run the following command
# Run SEEDLab VM which has Apache server running
! sysctl -q net.ipv4.tcp_max_syn_backlog

net.ipv4.tcp_max_syn_backlog = 128


In [60]:
# Turn of SYN cookie countermeasure
! sudo sysctl -a | grep cookie #(Display the SYN cookie flag)

net.ipv4.tcp_syncookies = 1


In [61]:
! sysctl -w net.ipv4.tcp_syncookies=0 #(turn off SYN cookie)

net.ipv4.tcp_syncookies = 0


In [43]:
# the following command shows all the halfopen connections
! netstat -na | grep SYN-RECV

In [66]:
# send SYN PACKET To the server and check for SYN-RECV status
pkt = IP(dst="192.168.100.5")/TCP(dport=80)
reply = sr1(pkt)

Begin emission:
Finished sending 1 packets.

Received 1 packets, got 1 answers, remaining 0 packets


In [68]:
reply.show()

###[ IP ]### 
  version= 4
  ihl= 5
  tos= 0x0
  len= 44
  id= 0
  flags= DF
  frag= 0
  ttl= 64
  proto= tcp
  chksum= 0xf171
  src= 192.168.100.5
  dst= 192.168.100.4
  \options\
###[ TCP ]### 
     sport= http
     dport= ftp_data
     seq= 3298356068
     ack= 1
     dataofs= 6
     reserved= 0
     flags= SA
     window= 29200
     chksum= 0x2c49
     urgptr= 0
     options= [('MSS', 1460)]
###[ Padding ]### 
        load= '\x00\x00'



In [69]:
# Send Get request
host = "192.168.100.5"
getStr = 'GET / HTTP1.1\r\nHost:{}\r\n\r\n'.format(host)
request = IP(dst=host) / TCP(dport=80, sport=syn_ack[TCP].dport, 
                                        seq=syn_ack[TCP].ack, ack=syn_ack[TCP].seq+1, 
                                        flags='A') / getStr
reply = sr1(request)

Begin emission:
Finished sending 1 packets.

Received 2 packets, got 1 answers, remaining 0 packets


## Create ping request
- IP packet with ICMP echo request and some payload
- send the packet and capture it with wireshark

In [9]:
ping = IP(src="192.168.1.145", dst="192.168.1.142") / ICMP() / "Hey there, are you up?"

In [10]:
ping.show()

###[ IP ]### 
  version= 4
  ihl= None
  tos= 0x0
  len= None
  id= 1
  flags= 
  frag= 0
  ttl= 64
  proto= icmp
  chksum= None
  src= 192.168.1.145
  dst= 192.168.1.142
  \options\
###[ ICMP ]### 
     type= echo-request
     code= 0
     chksum= None
     id= 0x0
     seq= 0x0
###[ Raw ]### 
        load= 'Hey there, are you up?'



In [11]:
send(ping)


Sent 1 packets.


In [6]:
reply = sr1(ping)
reply.show()

Begin emission:
Finished sending 1 packets.

Received 1 packets, got 1 answers, remaining 0 packets
###[ IP ]### 
  version= 4
  ihl= 5
  tos= 0x0
  len= 50
  id= 41200
  flags= 
  frag= 0
  ttl= 57
  proto= icmp
  chksum= 0x8653
  src= 93.184.216.34
  dst= 192.168.100.4
  \options\
###[ ICMP ]### 
     type= echo-reply
     code= 0
     chksum= 0x4c09
     id= 0x0
     seq= 0x0
###[ Raw ]### 
        load= 'Hey there, are you up?'



In [7]:
reply['Raw']

<Raw  load='Hey there, are you up?' |>

## Exercise
- Spoof src IP and ping some IP in research/controlled network
- 